In [1]:
import numpy as np
import os
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import tarfile

In [3]:
file = tarfile.open('aclImdb_v1.tar.gz')
file.extractall()

In [8]:
ROOT = '/content/aclImdb/train/pos'

In [12]:
reviews = []
for file in os.listdir(ROOT):
    path = os.path.join(ROOT, file)
    if os.path.isfile(path):
        with open(path, 'r') as fin:
            reviews.append(fin.read())

In [13]:
len(reviews)

12500

In [17]:
vect = TfidfVectorizer(stop_words='english')
x = vect.fit_transform(reviews)
pd.DataFrame(x.toarray(),columns=vect.get_feature_names())


,00,000,000s,003830,006,007,0079,0080,0083,0093638,00am,00pm,00s,01,02,020410,03,04,05,06,06th,07,08,089,08th,09,0ne,0r,10,100,1000,10000000000000,1000s,1001,100k,100m,100s,100th,101,101st,...,zp,zsigmond,zu,zubeidaa,zucker,zuckerman,zuckers,zucovic,zues,zuf,zugsmith,zukor,zukovic,zulu,zuniga,zuzz,zvonimir,zy,zé,álex,álvaro,ángel,ángela,âme,är,ääliöt,äänekoski,åge,æsthetic,écran,élan,émigré,émigrés,était,état,étc,êxtase,ís,østbye,über
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.054065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
topics = 15
nmf = NMF(n_components=topics)
W = nmf.fit_transform(x)
H = nmf.components_

In [23]:
words = np.array(vect.get_feature_names())
topic_words = pd.DataFrame(np.zeros((topics, 10)), index=[f'Topic {i + 1}' for i in range(topics)],
                           columns=[f'Word {i + 1}' for i in range(10)]).astype(str)
for i in range(topics):
    ix = H[i].argsort()[::-1][:10]
    topic_words.iloc[i] = words[ix]

topic_words

,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10
Topic 1,br,10,ll,spoilers,end,simply,yes,just,spoiler,quite
Topic 2,movie,movies,watch,recommend,10,saw,actors,acting,definitely,enjoyed
Topic 3,film,films,characters,director,cinema,festival,work,art,scenes,acting
Topic 4,series,episode,episodes,season,tv,characters,trek,seasons,shows,television
Topic 5,life,story,family,real,young,father,people,characters,lives,way
Topic 6,good,pretty,story,bad,acting,really,job,liked,nice,little
Topic 7,war,world,documentary,american,people,history,soldiers,men,hitler,women
Topic 8,man,role,performance,character,best,john,cast,actor,plays,played
Topic 9,like,really,think,just,don,people,know,say,didn,lot
Topic 10,seen,ve,time,years,saw,dvd,version,book,best,tv


In [24]:
topic_mapping = {
    'Topic 4': 'TV',
    'Topic 7': 'War',
    'Topic 8': 'Comedy',
    'Topic 12': 'Book Adaptation',
    'Topic 13': 'Horror',
    'Topic 15': 'Martial Arts / Action'
}

In [25]:
W = pd.DataFrame(W, columns=[f'Topic {i + 1}' for i in range(topics)])
W['max_topic'] = W.apply(lambda x: topic_mapping.get(x.idxmax()), axis=1)
W[pd.notnull(W['max_topic'])].head(10)

,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9,Topic 10,Topic 11,Topic 12,Topic 13,Topic 14,Topic 15,max_topic
0,0.000198,0.007691,0.001912,0.000547,0.009062,0.000136,0.007268,0.009191,0.001341,0.000000,0.000846,0.022060,0.000716,0.000000,0.000000,Book Adaptation
3,0.000011,0.000000,0.001044,0.015087,0.002055,0.000000,0.034061,0.015223,0.012592,0.000000,0.000000,0.000000,0.000000,0.000000,0.004274,War
12,0.000000,0.000000,0.000000,0.013783,0.000000,0.000000,0.052539,0.000000,0.000000,0.024956,0.000000,0.000000,0.000000,0.000000,0.000000,War
16,0.018377,0.004418,0.011447,0.001714,0.001110,0.006886,0.000000,0.019617,0.002574,0.000000,0.004571,0.001798,0.006520,0.003025,0.005042,Comedy
23,0.010896,0.012963,0.003998,0.001817,0.004408,0.000000,0.005859,0.031149,0.000000,0.000000,0.000000,0.000000,0.000000,0.006317,0.000000,Comedy
26,0.000000,0.000000,0.000000,0.001379,0.008843,0.000000,0.000000,0.010973,0.018094,0.000000,0.000000,0.037520,0.001551,0.000000,0.000000,Book Adaptation
28,0.000101,0.000000,0.000000,0.000272,0.005284,0.020635,0.000000,0.000000,0.011766,0.000000,0.000000,0.000000,0.000000,0.000141,0.028722,Martial Arts / Action
30,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024684,0.000000,0.000000,0.000000,0.004671,0.019603,0.008380,0.000000,Comedy
33,0.000000,0.002655,0.004982,0.026579,0.000000,0.003859,0.000000,0.027158,0.001192,0.003000,0.000000,0.000000,0.000000,0.007483,0.000000,Comedy
34,0.000279,0.046704,0.000000,0.000000,0.000000,0.059859,0.000000,0.000000,0.004292,0.020303,0.000000,0.003283,0.000000,0.007376,0.090268,Martial Arts / Action


In [29]:
reviews[26]

"*What I Like About SPOILERS* Teenager Holly Tyler (Amanda Bynes) goes to live with older sister Valerie (Jennie Garth) to avoid moving to Japan with her father; but she doesn't know the half of the wacky things that will happen to her from now on, and not only to her, but to her sister, her friends Gary (Wesley Jonathan) and Tina (Alison Munn), boyfriend Henry (Michael McMillian), crush Vince (Nick Zano), Valerie's boyfriend Jeff (Simon Rex), first boss (then firefighter then husband) Vic (Dan Cortese), annoying colleague Lauren (Leslie Grossman) and second boss Peter (?) If you don't have a funny bone in your body, please skip this; if you like only veeeery sophisticated comedy this isn't for you; if you like a funny, sometimes touching show with two hot chicks who can act in the lead (and none other than the fabulous 'Mary Cherry' from Popular - Leslie Grossman - in the main cast), then what the hell are you waiting for? You're welcome to Casa De Tyler! What I Like About You (2002-2